# Table of Contents
1. [Import Libraries](#1)<br>
2. [Conv Block Function](#2)<br>
3. [Encoder and Decoder Function](#3)<br>
4. [Build Unet Function](#4)<br>
5. [Dice Coef and Dice Loss Functions](#5)<br>
6. [Data Loader Function](#6)<br>
7. [Data Generator Function](#7)<br>
8. [Prepare data paths](#8)<br>
9. [Complie and Fit the Model ¶](#9)<br>
10. [Test Model](#10)<br>

# 1. Import Libraries <a id = "1"><a>

In [1]:
import nibabel as nib
import time
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import cv2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# pip install nibabel

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dropout, BatchNormalization, UpSampling2D,Activation, MaxPool2D,MaxPooling2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

# from unet import build_unet
# from metrics import dice_loss, dice_coef

In [4]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import data
from skimage.util import montage 
import skimage.transform as skTrans
from skimage.transform import rotate
from skimage.transform import resize
from PIL import Image, ImageOps  


# neural imaging
import nilearn as nl
import nibabel as nib
import nilearn.plotting as nlplt
!pip install git+https://github.com/miykael/gif_your_nifti # nifti to gif 
import gif_your_nifti.core as gif2nif


# ml libs
import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

  Cloning https://github.com/miykael/gif_your_nifti to /tmp/pip-req-build-80_6g27a
  Running command git clone --filter=blob:none --quiet https://github.com/miykael/gif_your_nifti /tmp/pip-req-build-80_6g27a
  Resolved https://github.com/miykael/gif_your_nifti to commit 9326cae8596805afd747ba3700b3032ece423c62
  Preparing metadata (setup.py) ... done
  Created wheel for gif-your-nifti: filename=gif_your_nifti-0.2.0-py3-none-any.whl size=6270 sha256=5a141fd5a502a7e03778e45c3dcc6ff5bbb000e0f6ca2686c67426e7018a0eca
  Stored in directory: /tmp/pip-ephem-wheel-cache-qw_3i38c/wheels/4a/8c/d1/b228c3b67231f7459e8f70d73f4dadaf65cd90692d41f43e88
Successfully built gif-your-nifti


# Read and show one image

In [5]:
# img = nib.load('/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t2.nii')
# img_array = img.get_fdata()
# print(img_array.shape)
# plt.figure(figsize=(20,20))
# for i in range(20):
#     plt.subplot(4,5,i+1)
#     plt.imshow(img_array[:,:,i], cmap='gray')


# Read and show one Mask

In [ ]:
# mask = nib.load('/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_seg.nii')
# mask_array = mask.get_fdata()
# print(mask_array.shape)
# plt.figure(figsize=(20,20))
# # for i in range(20):
# #     plt.subplot(4,5,i+1)
# #     plt.imshow(mask_array[:,:,i], cmap='gray')
# #     plt.colorbar()
# print(np.unique(mask_array))

# Read and Load first 10 patients images

In [ ]:
# start_time = time.time()
# image_slices_list = []
# for i in range(1,11):
#     if i <10:
#         img = nib.load(f"/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_00{i}/BraTS20_Training_00{i}_flair.nii")
#     else:
#         img = nib.load(f"/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_0{i}/BraTS20_Training_0{i}_flair.nii")
#     img_arr = img.get_fdata()
#     for i in range(155):
#         RGB_img = np.repeat(img_arr[:,:,i][...,np.newaxis], 3, axis=-1)
#         image_slices_list.append(RGB_img)
        
        
# image_slices_arr = np.array(image_slices_list)
# end_time = time.time()
# print(f"Execution Time: {(end_time-start_time)//60} Min")
# print(f"Shape of image_slices_arr: {image_slices_arr.shape}")

# Read and Load first 10 patients masks

In [ ]:
# start_time = time.time()
# masks_slices_list = []
# masks_slices_list_1_hot = []

# for i in range(1,11):
#     if i <10:
#         mask = nib.load(f"/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_00{i}/BraTS20_Training_00{i}_seg.nii")
#     else:
#         mask = nib.load(f"/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_0{i}/BraTS20_Training_0{i}_seg.nii")
#     mask_arr = mask.get_fdata()
#     for i in range(155):
#         mask = mask_arr[:,:,i]
#         mask[mask==4] = 3
#         masks_slices_list.append(mask)
#         mask_1_hot =  to_categorical(mask, num_classes=4)
#         masks_slices_list_1_hot.append(mask_1_hot)
        
# masks_slices_arr = np.array(masks_slices_list)
# masks_slices_arr_1_hot = np.array(masks_slices_list_1_hot)

# end_time = time.time()
# print(f"Execution Time: {(end_time-start_time)//60} Min")
# print(f"Shape of masks_slices_arr: {masks_slices_arr.shape}")
# print(f"Shape of masks_slices_list_1_hot: {masks_slices_arr_1_hot.shape}")

# 2. Conv Block Function <a id = "" ><a>

In [6]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

# 3. Encoder and Decoder Function <a id = "3" ><a>

In [7]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

# 4. Build U-Net Function <a id = "4" ><a>

In [8]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # print(s1.shape, s2.shape, s3.shape, s4.shape)
    # print(p1.shape, p2.shape, p3.shape, p4.shape)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(4, (1,1), padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

# 5. Dice Coef and Dice Loss Functions <a id = "5"><a>

In [9]:
# dice loss as defined above for 4 classes
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 4
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
   #     K.print_tensor(loss, message='loss value for class {} : '.format(SEGMENT_CLASSES[i]))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
    total_loss = total_loss / class_num
#    K.print_tensor(total_loss, message=' total dice coef: ')
    return total_loss


 
# define per class evaluation of dice coef
# inspired by https://github.com/keras-team/keras/issues/9395
def dice_coef_necrotic(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,1] * y_pred[:,:,:,1]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,1])) + K.sum(K.square(y_pred[:,:,:,1])) + epsilon)

def dice_coef_edema(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,2] * y_pred[:,:,:,2]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,2])) + K.sum(K.square(y_pred[:,:,:,2])) + epsilon)

def dice_coef_enhancing(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,3] * y_pred[:,:,:,3]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,3])) + K.sum(K.square(y_pred[:,:,:,3])) + epsilon)



# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    
# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [11]:
smooth = 1e-15
def dice_coef(y_true, y_pred, smooth=100):        
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# 6. Data Loader Functions <a id = "6"><a>

In [14]:
def data_loader_v2(images_path):
    images = []
    masks = []
#     print("ENTERED DATA LOADER")
    for i in range(len(images_path)):
            patient_num = images_path[i].split('/')[-1]
            data_path = os.path.join(images_path[i], f'{patient_num}_flair.nii');
            flair = nib.load(data_path).get_fdata()    

            data_path = os.path.join(images_path[i], f'{patient_num}_t1ce.nii');
            t1ce = nib.load(data_path).get_fdata()
            
            data_path = os.path.join(images_path[i], f'{patient_num}_t1.nii');
            t1 = nib.load(data_path).get_fdata()
            
            data_path = os.path.join(images_path[i], f'{patient_num}_seg.nii');
            mask = nib.load(data_path).get_fdata()
            
            for i in range(155): #iterate over all 155 slices and remove black slices
                flt_mask = np.array(mask[:,:,i]).flatten()
                flag =  all(pixel == flt_mask[0] for pixel in flt_mask)
                if not flag:    
                    resized_mask =cv2.resize(mask[:,:,i],(128,128))
                    resized_mask[resized_mask==4] = 3
                    mask_1_hot =  to_categorical(resized_mask, num_classes=4)
                    masks.append(mask_1_hot)
                    
                    resized_img_t1 =cv2.resize(t1[:,:,i],(128,128))
                    resized_img_t1ce =cv2.resize(t1ce[:,:,i],(128,128))
                    resized_img_flair =cv2.resize(flair[:,:,i],(128,128))
                    img = np.stack((resized_img_t1,resized_img_t1ce,resized_img_flair),axis=-1)
                    images.append(img)
                else:
                    continue
 
    return np.array(images) , np.array(masks) 

# 7. Data Generator Functions <a id = "7"><a>

In [16]:
def data_generator(images_paths_list, batch_size):
    L = len(images_paths_list)
    while True:
        
        batch_start = 0
        batch_end = batch_size
        
        while batch_start < L:
            
            limit = min(batch_end,L)
            x , y = data_loader_v2(images_paths_list[batch_start:limit])  
            
            yield(x,y)
            
            batch_start += batch_size
            batch_end +=batch_size
#             print("Shape of batch_images form Data Gen",np.array(x).shape)
#             print("Shape of batch_masks form Data Gen",np.array(y).shape)

# 8. Prepare data paths <a id = "8"><a>

In [20]:
TRAIN_DATASET_PATH =r'/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
train_and_val_directories = [f.path for f in os.scandir(TRAIN_DATASET_PATH) if f.is_dir()]
train_and_val_directories = sorted(train_and_val_directories)
len(train_and_val_directories)
print(train_and_val_directories[:10],sep="\n")
train_and_val_directories.remove(train_and_val_directories[354])


['/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001', '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_002', '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_003', '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_004', '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_005', '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_006', '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_007', '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS2

In [24]:
train_test_images_paths, val_images_paths = train_test_split(train_and_val_directories,test_size=0.2) #split to .... and val 
train_images_paths, test_images_paths = train_test_split(train_test_images_paths,test_size=0.15) #split to train and test 

print(f"Shape of train_images_paths: {np.array(train_images_paths).shape}")
print(f"Shape of val_images_paths: {np.array(val_images_paths).shape}")
print(f"Shape of test_images_paths: {np.array(test_images_paths).shape}")


Shape of train_images_paths: (249,)
Shape of val_images_paths: (74,)
Shape of test_images_paths: (45,)


In [25]:
input_shape = (128, 128, 3)


# 9. Complie and Fit the Model <a id = "9"><a>

In [26]:
model = build_unet(input_shape)
model.compile(loss=dice_loss, optimizer=Adam(learning_rate=0.0001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), 
                                            dice_coef, precision, sensitivity, specificity, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )
BATCH_SIZE = 1
EPOCHS= 50

callbacks = [
#         ModelCheckpoint(model, verbose=1, save_best_only=True),
#         ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
#         CSVLogger(csv_path),
#         EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
            ]
model.fit(data_generator(train_images_paths, 1),
#           validation_data =(data_generator(val_images_paths ,1)),
#         validation_data=(val_images,val_masks),
        epochs=EPOCHS,
        batch_size= BATCH_SIZE,
        steps_per_epoch = 20,
#         callbacks=callbacks
        )

2023-01-11 19:52:53.310977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 19:52:53.415190: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 19:52:53.415977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-11 19:52:53.417154: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/50


2023-01-11 19:53:03.648855: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


20/20 [==============================] - 77s 3s/step - loss: 0.5424 - accuracy: 0.7784 - mean_io_u: 0.3750 - dice_coef: 0.4585 - precision: 0.6874 - sensitivity: 0.7978 - specificity: 0.8784 - dice_coef_necrotic: 0.0513 - dice_coef_edema: 0.0741 - dice_coef_enhancing: 0.0106
Epoch 2/50
20/20 [==============================] - 61s 3s/step - loss: 0.4281 - accuracy: 0.9078 - mean_io_u: 0.3751 - dice_coef: 0.5705 - precision: 0.8525 - sensitivity: 0.9170 - specificity: 0.9468 - dice_coef_necrotic: 0.0867 - dice_coef_edema: 0.1366 - dice_coef_enhancing: 0.0084
Epoch 3/50
20/20 [==============================] - 60s 3s/step - loss: 0.3805 - accuracy: 0.9679 - mean_io_u: 0.3754 - dice_coef: 0.6187 - precision: 0.9407 - sensitivity: 0.9769 - specificity: 0.9794 - dice_coef_necrotic: 0.1306 - dice_coef_edema: 0.2064 - dice_coef_enhancing: 0.0095
Epoch 4/50
20/20 [==============================] - 48s 2s/step - loss: 0.3558 - accuracy: 0.9678 - mean_io_u: 0.3753 - dice_coef: 0.6427 - precision:

In [ ]:
model.save('./unet_2d.h5')

In [ ]:
from IPython.display import FileLink
FileLink("./unet_2d.h5")

# 10. Test Model <a id ="10"><a>

In [27]:
train_images ,train_true_masks = data_loader_v2(train_images_paths[7 : 8] )
test_images ,test_true_masks = data_loader_v2(test_images_paths[7:8] )

In [ ]:
print(train_images.shape)
print(train_true_masks.shape)
print(test_images.shape)
print(test_true_masks.shape)

In [28]:
train_scores = model.evaluate(train_images, train_true_masks, verbose=0)
test_scores = model.evaluate(test_images, test_true_masks, verbose=0)
# validation_scores = model_cnn2.evaluate(X_valid, y_valid, verbose=0)

print(f"Train scores: {model.metrics_names[0]}: {train_scores[0]} , {model.metrics_names[1]}: {train_scores[1]}")
print(f"Test scores: {model.metrics_names[0]}: {test_scores[0]} , {model.metrics_names[1]}: {test_scores[1]}")

Train scores: loss: 0.08163738250732422 , accuracy: 0.9793108701705933
Test scores: loss: 0.09821661561727524 , accuracy: 0.9584391117095947
